In [1]:
import torch
print(torch.cuda.is_available())

True


In [2]:
from data_logger import DataLogger
from dqn_agent import SonicAgent 
from utils import *

logger = DataLogger(model="DDQN")

In [3]:
import retro
episodes = 10
frame_shape = (84, 84)
env = retro.make(game="SonicTheHedgehog-Genesis", state="GreenHillZone.Act1")

sonic_agent = SonicAgent(observation_size=(4,)+frame_shape, action_size=len(POSSIBLE_ACTIONS), exploration_rate=1, discount_factor=0.995)#initial value for epsilon=1
env = apply_wrappers(env, skip=4, shape=frame_shape, num_stack=4)

for episode in range(episodes):
    print(f"---Episode {episode}")
    state = env.reset()
    time_stamp = 0
    prev_live = 3
    while time_stamp<1e4:
        action = sonic_agent.act(state)

        next_state, reward, done, info = env.step(POSSIBLE_ACTIONS[action])

        sonic_agent.append_experience_to_memory(state, next_state, action, reward, done)
        
        loss = sonic_agent.learn()    

        logger.add(loss, sonic_agent.step)

        state = next_state

        env.render()
        
        time_stamp += 1

        lost_life = (info["lives"]!=prev_live)
        prev_live = info["lives"]

        if done or lost_life:
            break
        

logger.close()

---Episode 0
---Episode 1


KeyboardInterrupt: 

In [6]:
env.close()